<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
from math import exp
from numpy.random import rand
import random
from lib.entities import State
from lib.operations import Dataset

In [2]:
dataset = Dataset()
with open("./data/input/simple_input.json", 'r') as f:
    dataset.read_json(f)

In [3]:
len(dataset.couriers)

2

In [4]:
def gibbs_probability(energy_old, energy_new, temp):
    if energy_old > energy_new:
        return 1
    else:
        return exp(-1.0 * (energy_old - energy_new) / temp)

def trivial_temperature(t):
    return t


def trivial_energy(state):
    return 100.


def trivial_probability(energy_before, energy_after, temp):
    return 1.


class TrivialState:
    def __init__(self, p):
        self.param = p
    
    def get_neighbour(self):
        return TrivialState(self.param + 1)

In [5]:
class SimmulatedAnnealing(object):
    # TODO: add default parameters
    def __init__(self, temperature_func, probability_func):
        """
        temperature_func(param) where param belongs to [0, 1]
        energy_func(state)
        probability_func()
        """
        self.temperature = temperature_func
        self.probability = probability_func
    
    def score(self, state):
        return state.max_money - state.energy()
    
    def fit_predict(self, initial_state, niter=15000):
        """ Returns final state after processing of niter iterations """
        temp = np.inf
        state = initial_state
        
        for iter in range(1, niter + 1):
            temp = self.temperature(1. * (niter + 1 - iter) / niter * state.max_money / 100.0)
            new_state = state.get_neighbour()
            
            if self.probability(state.energy(), new_state.energy(), temp) > rand():
                state = new_state
            if iter % 1000 == 0:
                print("Completed", iter, "iterations", "score =", self.score(new_state))
                
        return state

In [6]:
model = SimmulatedAnnealing(trivial_temperature, gibbs_probability)

In [7]:
%%time

initial_state = State(dataset) # make something smart

ans_state = model.fit_predict(initial_state, niter=1000)

Completed 1000 iterations score = 1364.0
CPU times: user 7.87 s, sys: 1.16 ms, total: 7.87 s
Wall time: 7.87 s


In [8]:
ans_state.max_money - ans_state.energy()

1364.0

In [9]:
ans_state.max_money

1800

In [11]:
with open("data/output/simple_output.json", 'w') as fd:
    ans_state.convert_to_json(fd)